In [1]:
import sys
import os
# when python interpreter is different, add path
gems_tco_path = "/Users/joonwonlee/Documents/GEMS_TCO-1/src"
sys.path.append(gems_tco_path)
import matplotlib.pyplot as plt

# Data manipulation and analysis
import pandas as pd
import numpy as np
import pickle 
from collections import defaultdict

from pathlib import Path
import time
import json
from json import JSONEncoder

# Special functions and optimizations
from typing import Callable, Union, Tuple
from scipy.spatial.distance import cdist  # For space and time distance
from scipy.special import gamma, kv  # Bessel function and gamma function
from scipy.interpolate import splrep, splev

import torch
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
from torchcubicspline import natural_cubic_spline_coeffs, NaturalCubicSpline

import GEMS_TCO
from GEMS_TCO import kernels 
from GEMS_TCO import orderings as _orderings
from GEMS_TCO import load_data

from GEMS_TCO import configuration as config


In [3]:


np.ceil(4/3)

2.0

In [2]:
# conda activate faiss_env

!/opt/anaconda3/envs/faiss_env/bin/python /Users/joonwonlee/Documents/GEMS_TCO-1/src/GEMS_TCO/mymac_config.py --space "20,20" --days "0,31"


/opt/anaconda3/envs/faiss_env/bin/python: can't open file '/Users/joonwonlee/Documents/GEMS_TCO-1/src/GEMS_TCO/mymac_config.py': [Errno 2] No such file or directory


In [2]:
lat_lon_resolution = [8,8]
years = ['2024']
month_range =[7,8]
nheads = 10
mm_cond_number = 10 
v= 0.5


data_load_instance = load_data(config.mac_data_load_path)
df = data_load_instance.read_pickle(config.mac_estimates_day_path,config.mac_full_day_v05_pickle)
map, ord_mm, nns_map= data_load_instance.load_mm20k_data_bymonthyear( lat_lon_resolution= lat_lon_resolution, mm_cond_number=mm_cond_number,years_=years, months_=month_range)

df.head()
days = list(range(0,2))

for day in days:
    params = list(df.iloc[day-1][:-1])
    params = torch.tensor(params, dtype=torch.float64, requires_grad=True)
    print(f'2024-07-{day+1}, data size per hour: { (int(158.7 / lat_lon_resolution[0] * (113.63 / lat_lon_resolution[0]))) }, smooth: {v}')

    idx_for_datamap= [ 8*(day),8*(day+1)]
    analysis_data_map, aggregated_data = data_load_instance.load_working_data_byday( map, ord_mm, nns_map, idx_for_datamap= idx_for_datamap)


2024-07-1, data size per hour: 281, smooth: 0.5
2024-07-2, data size per hour: 281, smooth: 0.5


For resolution 8,8, compare full vs spline vecc vs vecc

In [ ]:
lat_lon_resolution = [8,8]
years = ['2024']
month_range =[7,8]
nheads = 10
mm_cond_number = 10 
v= 0.5

data_load_instance = load_data(config.mac_data_load_path)
df = data_load_instance.read_pickle(config.mac_estimates_day_path,config.mac_full_day_v05_pickle)
map, ord_mm, nns_map= data_load_instance.load_mm20k_data_bymonthyear( lat_lon_resolution= lat_lon_resolution, mm_cond_number=mm_cond_number,years_=years, months_=month_range)

df.head()
days = list(range(1,31))
for day in days:
    params = list(df.iloc[day-1][:-1])
    params = torch.tensor(params, dtype=torch.float64, requires_grad=True)
    print(f'2024-07-{day+1}, data size per hour: { (int(158.7 / lat_lon_resolution[0] * (113.63 / lat_lon_resolution[0]))) }, smooth: {v}')

    idx_for_datamap= [ 8*(day),8*(day+1)]
    analysis_data_map, aggregated_data = data_load_instance.load_working_data_byday( map, ord_mm, nns_map, idx_for_datamap= idx_for_datamap)


   
    for ccc in [3,4,6]:
        coarse_factor_head = 4 # 2:64 4:16 8(284):4, 16(70):1
        coarse_factor_cond = 1 # conservatove 4: 12,14
        spline_instance = kernels.spline(epsilon = 0, coarse_factor= ccc, nheads = 80, smooth = 0.5, input_map= analysis_data_map, aggregated_data= aggregated_data, nns_map=nns_map, mm_cond_number=10)

        distances, _ = spline_instance.precompute_coords_anisotropy(params, aggregated_data, aggregated_data)
        spline_instance.spline_object = spline_instance.fit_cubic_spline( distances, spline_instance.coarse_factor)  # change here


        cov_map_spline = spline_instance.cov_structure_saver_using_spline(params)
        vecc = spline_instance.vecchia_nll_using_spline(params, cov_map_spline)


        instance = kernels.vecchia_experiment(v, analysis_data_map, aggregated_data,nns_map,mm_cond_number, nheads)

        cov_map = instance.cov_structure_saver(params, instance.matern_cov_anisotropy_kv)
        vecc_ll = instance.vecchia_may9(params, instance.matern_cov_anisotropy_kv ,cov_map)

        instance = kernels.vecchia_experiment(0.5, analysis_data_map, aggregated_data,nns_map,mm_cond_number, nheads)
        excat_ll = instance.full_likelihood(params, aggregated_data[:,:4], aggregated_data[:,2], instance.matern_cov_anisotropy_v05)

        print(f'coarse_fact0r: {ccc} full: {excat_ll.item()}, vecc: {vecc_ll.item()} , spline vecc: {vecc.item()}')   # (14435.97)

2024-07-2, data size per hour: 281, smooth: 0.5
tensor(45.6056, dtype=torch.float64)
coarse_fact0r: 3 full: 4180.555600142574, vecc: 4252.477514877542 , spline vecc: 4233.13839188455
tensor(45.6056, dtype=torch.float64)
coarse_fact0r: 4 full: 4180.555600142574, vecc: 4252.477514877542 , spline vecc: 4234.006403046005
tensor(45.6056, dtype=torch.float64)
coarse_fact0r: 6 full: 4180.555600142574, vecc: 4252.477514877542 , spline vecc: 4223.168181158035
2024-07-3, data size per hour: 281, smooth: 0.5
tensor(33.8880, dtype=torch.float64)
coarse_fact0r: 3 full: 5366.250278615454, vecc: 5385.952115700239 , spline vecc: 5384.270413851868
tensor(33.8880, dtype=torch.float64)
coarse_fact0r: 4 full: 5366.250278615454, vecc: 5385.952115700239 , spline vecc: 5385.69697437473
tensor(33.8880, dtype=torch.float64)
coarse_fact0r: 6 full: 5366.250278615454, vecc: 5385.952115700239 , spline vecc: 5383.261958928432
2024-07-4, data size per hour: 281, smooth: 0.5
tensor(63.7845, dtype=torch.float64)
coars

For resolution (4,4) compare vecc vs spline vecc

In [ ]:
lat_lon_resolution = [4,4]
years = ['2024']
month_range =[7,8]
nheads = 10
mm_cond_number = 10 
v= 0.5


data_load_instance = load_data(config.mac_data_load_path)
df = data_load_instance.read_pickle(config.mac_estimates_day_path,config.mac_full_day_v05_pickle)
map, ord_mm, nns_map= data_load_instance.load_mm20k_data_bymonthyear( lat_lon_resolution= lat_lon_resolution, mm_cond_number=mm_cond_number,years_=years, months_=month_range)

df.head()
days = list(range(0,15))

for day in days:
    params = list(df.iloc[day-1][:-1])
    params = torch.tensor(params, dtype=torch.float64, requires_grad=True)
    print(f'2024-07-{day+1}, data size per hour: { (int(158.7 / lat_lon_resolution[0] * (113.63 / lat_lon_resolution[0]))) }, smooth: {v}')

    idx_for_datamap= [ 8*(day),8*(day+1)]
    analysis_data_map, aggregated_data = data_load_instance.load_working_data_byday( map, ord_mm, nns_map, idx_for_datamap= idx_for_datamap)

    for ccc in [12,16]:

        coarse_factor_head = ccc # 2:64 4:16 8(284):4, 16(70):1
        coarse_factor_cond = 1 # conservatove 4: 12,14
        spline_instance = kernels.spline(epsilon = 0, coarse_factor = coarse_factor_head, nheads = 280, smooth = 0.5, input_map= analysis_data_map, aggregated_data= aggregated_data, nns_map=nns_map, mm_cond_number=10)

        spline_instance.nheads= 280

        distances, _ = spline_instance.precompute_coords_anisotropy(params, aggregated_data, aggregated_data)
        spline_instance.spline_object = spline_instance.fit_cubic_spline( distances, spline_instance.coarse_factor)  # change here
  
        cov_map_spline = spline_instance.cov_structure_saver_using_spline(params)
        vecc = spline_instance.vecchia_nll_using_spline(params, cov_map_spline)

        instance = kernels.vecchia_experiment(v, analysis_data_map, aggregated_data,nns_map,mm_cond_number, nheads)

        cov_map = instance.cov_structure_saver(params, instance.matern_cov_anisotropy_kv)
        vecc_ll = instance.vecchia_may9(params, instance.matern_cov_anisotropy_kv ,cov_map)
        print(f'coarse_fact0r: {ccc} vecc: {vecc_ll.item()}, spline vecc: {vecc.item()}')   # (14435.97)

2024-07-1, data size per hour: 1127, smooth: 0.5
tensor(19.7589, dtype=torch.float64)
coarse_fact0r: 12 vecc: 15003.005412067303, spline vecc: 14975.037920147433
tensor(19.7589, dtype=torch.float64)
coarse_fact0r: 16 vecc: 15003.005412067303, spline vecc: 14973.03303072827
2024-07-2, data size per hour: 1127, smooth: 0.5
tensor(48.0573, dtype=torch.float64)
coarse_fact0r: 12 vecc: 14615.103575022533, spline vecc: 14562.541601918481
tensor(48.0573, dtype=torch.float64)
coarse_fact0r: 16 vecc: 14615.103575022533, spline vecc: 14552.964569265665
2024-07-3, data size per hour: 1127, smooth: 0.5
tensor(35.6930, dtype=torch.float64)
coarse_fact0r: 12 vecc: 21023.250809659956, spline vecc: 20915.435910790955
tensor(35.6930, dtype=torch.float64)
coarse_fact0r: 16 vecc: 21023.250809659956, spline vecc: 20919.28010351776
2024-07-4, data size per hour: 1127, smooth: 0.5
tensor(67.1735, dtype=torch.float64)
coarse_fact0r: 12 vecc: 14226.09708978822, spline vecc: 14215.100450398431
tensor(67.1735, 

For resolution 2,2

In [ ]:
lat_lon_resolution = [2,2]
years = ['2024']
month_range =[7,8]
nheads = 10
mm_cond_number = 10 
v= 0.5

data_load_instance = load_data(config.mac_data_load_path)
df = data_load_instance.read_pickle(config.mac_estimates_day_path,config.mac_full_day_v05_pickle)
map, ord_mm, nns_map= data_load_instance.load_mm20k_data_bymonthyear( lat_lon_resolution= lat_lon_resolution, mm_cond_number=mm_cond_number,years_=years, months_=month_range)

df.head()
days = list(range(0,31))

for day in days:
    params = list(df.iloc[day-1][:-1])
    params = torch.tensor(params, dtype=torch.float64, requires_grad=True)
    print(f'2024-07-{day+1}, data size per hour: { (int(158.7 / lat_lon_resolution[0] * (113.63 / lat_lon_resolution[0]))) }, smooth: {v}')

    idx_for_datamap= [ 8*(day),8*(day+1)]
    analysis_data_map, aggregated_data = data_load_instance.load_working_data_byday( map, ord_mm, nns_map, idx_for_datamap= idx_for_datamap)

    instance = kernels.vecchia_experiment(v, analysis_data_map, aggregated_data,nns_map,mm_cond_number, nheads)

    cov_map = instance.cov_structure_saver(params, instance.matern_cov_anisotropy_kv)
    vecc_ll = instance.vecchia_may9(params, instance.matern_cov_anisotropy_kv ,cov_map)
    
    for ccc in [60]:

        coarse_factor_head = ccc # 2:64 4:16 8(284):4, 16(70):1
        coarse_factor_cond = 1 # conservatove 4: 12,14
        spline_instance = kernels.spline(epsilon = 0, coarse_factor = coarse_factor_head, nheads = 280, smooth = 0.5, input_map= analysis_data_map, aggregated_data= aggregated_data, nns_map=nns_map, mm_cond_number=10)

   

        distances, _ = spline_instance.precompute_coords_anisotropy(params, aggregated_data, aggregated_data)
        spline_instance.spline_object = spline_instance.fit_cubic_spline( distances, spline_instance.coarse_factor)  # change here

        cov_map_spline = spline_instance.cov_structure_saver_using_spline(params)
        vecc = spline_instance.vecchia_nll_using_spline(params, cov_map_spline)

        print(f'coarse_fact0r: {ccc} vecc: {vecc_ll.item()}, spline vecc: {vecc.item()}')   # (14435.97)

day 1 50: 55179, 55090      64:   55179  55013   50:55179 55035
day2 50:  56304  56679      64:   keep crashing with 64 and slower than 50 factor
day 3 50: 78034  81828  (1m 11)     60: 1m 15s  80149

Compare Full vs Spline full

In [9]:
coarse_factor_head = 4 # 16:2 8:4  4:16, i expect 2:64, 1:128
coarse_factor_cond = 1
v= 0.5
spline_instance = kernels.spline(epsilon = 0, coarse_factor=coarse_factor_head, nheads=80, smooth = v, input_map= analysis_data_map, aggregated_data= aggregated_data, nns_map=nns_map, mm_cond_number=10)
distances, non_zero_indices = spline_instance.precompute_coords_anisotropy(params, spline_instance.aggregated_data, spline_instance.aggregated_data)

distances, non_zero_indices = spline_instance.precompute_coords_anisotropy(params, aggregated_data, aggregated_data)
spline_object_head = spline_instance.fit_cubic_spline( distances, spline_instance.coarse_factor)  # change here

spline_full = spline_instance.full_likelihood_using_spline(params,aggregated_data[:,:4], aggregated_data[:,2], distances, spline_object_head)

instance = kernels.vecchia_experiment(0.5, analysis_data_map, aggregated_data,nns_map,mm_cond_number, nheads)
excat_ll = instance.full_likelihood(params, aggregated_data[:,:4], aggregated_data[:,2], instance.matern_cov_anisotropy_v05)

# 12663.4804
print( f'exact full: {excat_ll.item()}, spline_full: {spline_full.item()}' )


exact full: 4180.555600142574, spline_full: 4181.870527733848


likelihood 

# debug error when high resolution cov_1d returns nans
# Summary I have to make the element wise different smaller than 2.53e-7 to make
# likelihood difference smaller than 0.15

## I suggest 1000 for resolution 1250(4,4) and 5000 for (2,2) and 50,000 for (1,1)


resolution 3,3
10,000:   total diff   1.66       5.01e-9
100,000                868        2.6e-6

resolution 4,4  (160000**2/(10000**2)  1/256 from original)
#coarse factor 5 error coarse factor 10 okay
coarse_factor 100 took 18 sec       sum diff 0.167   1.67e-9
coarse_factor 1000 okay difference elementwise ( sum diff 0.2831, 2.83e-9 )
coarse_factor 10,000        sum difference 1.45 (   1.45/10000**2= 1.5e-8  )

resolution 6,6
100:     sum:0.028  1.315e-9
1000:   sum: 0.0314   1.47e-9
10000:  sum: -124    5.82e-6

resolution 8,8

coarse_factor 4     sum diff 0.02     8.5e-9
coarse_factor 1000    sum diff  -13.8154   -5.39 e-6

coarse_factor 10,000  sum diff 3793
coarse_factor 100,000 began to show difference at 10-4

resolution 20,20
coarse_facttor 100    sum diff 5.729   3.57e-5
coarse_factor 1000    sum diff  200

In [4]:
coarse_factor_head = 4 # 16:2 8:4  4:16, i expect 2:64, 1:128

spline_instance = kernels.spline(epsilon = 0, coarse_factor=coarse_factor_head, nheads=280, smooth = v, input_map= analysis_data_map, aggregated_data= aggregated_data, nns_map=nns_map, mm_cond_number=10)


distances, non_zero_indices = spline_instance.precompute_coords_anisotropy(params, aggregated_data, aggregated_data)


In [ ]:
spline_object_head = spline_instance.fit_cubic_spline( distances, spline_instance.coarse_factor)  # change here
cov_1d = spline_object_head.evaluate(distances)
sigmasq, _, _, _, _, _, nugget = params
cov_matrix = cov_1d.reshape(distances.shape)
cov_matrix = cov_matrix * sigmasq
cov_matrix = cov_matrix + torch.eye(cov_matrix.shape[0], dtype=torch.float64) * nugget 
print(f'Covarinace matrix using spline:\n\n {cov_matrix}\n')

instance_2 = kernels.vecchia_experiment(v, analysis_data_map, aggregated_data,nns_map,mm_cond_number, nheads)
out = instance_2.matern_cov_anisotropy_kv(params, instance_2.aggregated_data, instance_2.aggregated_data)
print(f'Original covarinace matrix : \n\n {out}')

print(f'shape of the matrices: {cov_matrix.shape}')

print(f'sum of differences: {torch.sum ( cov_matrix-out )}')
print(f'element-wise difference on average: {torch.sum(cov_matrix-out)/ cov_matrix.shape[0]**2}')

Covarinace matrix using spline:

 tensor([[25.9374,  1.3838,  1.7379,  ...,  4.9747,  2.3263,  7.1838],
        [ 1.3838, 25.9374,  0.1766,  ...,  3.6737,  0.1986,  2.2543],
        [ 1.7379,  0.1766, 25.9374,  ...,  0.7044,  7.3124,  1.1109],
        ...,
        [ 4.9747,  3.6737,  0.7044,  ..., 25.9374,  1.0808, 14.1243],
        [ 2.3263,  0.1986,  7.3124,  ...,  1.0808, 25.9374,  1.8244],
        [ 7.1838,  2.2543,  1.1109,  ..., 14.1243,  1.8244, 25.9374]],
       dtype=torch.float64, grad_fn=<AddBackward0>)

Original covarinace matrix : 

 tensor([[25.9374,  1.3838,  1.7379,  ...,  4.9747,  2.3263,  7.1838],
        [ 1.3838, 25.9374,  0.1766,  ...,  3.6737,  0.1986,  2.2543],
        [ 1.7379,  0.1766, 25.9374,  ...,  0.7044,  7.3124,  1.1109],
        ...,
        [ 4.9747,  3.6737,  0.7044,  ..., 25.9374,  1.0808, 14.1243],
        [ 2.3263,  0.1986,  7.3124,  ...,  1.0808, 25.9374,  1.8244],
        [ 7.1838,  2.2543,  1.1109,  ..., 14.1243,  1.8244, 25.9374]],
       dtype=

optimization for full likelihood

In [ ]:
coarse_factor_head = 4 # 16:2 8:4  4:16, i expect 2:64, 1:128
coarse_factor_cond = 1
spline_instance = kernels.spline(epsilon = 0, coarse_factor=coarse_factor_head, nheads= 50, smooth = v, input_map= analysis_data_map, aggregated_data= aggregated_data, nns_map=nns_map, mm_cond_number=10)

# print(params)

optimizer, scheduler = spline_instance.optimizer_fun(params, lr=0.02, betas=(0.9, 0.99), eps=1e-8, step_size=100, gamma=0.2)  
out, epoch = spline_instance.run_full(params, aggregated_data,optimizer,scheduler, epochs=500)


tensor([ 2.4793e+01,  1.5845e+00,  1.7182e+00,  9.0885e-03, -1.0730e-01,
         1.3104e-01,  2.7172e+00], dtype=torch.float64, requires_grad=True)
Epoch 1, Gradients: [-1.52004529e+01  1.20108038e+02 -9.76898219e-01  1.16965185e+02
  3.59766977e+02 -1.09880554e+03 -1.32770512e+02]
 Loss: 4181.870527733848, Parameters: [ 2.47934437e+01  1.58452892e+00  1.71824777e+00  9.08850413e-03
 -1.07299447e-01  1.31037638e-01  2.71723866e+00]
Epoch 101, Gradients: [ -1.12692246  -0.77355936   0.29054058   0.59414848  -1.13468292
   3.4382319  -14.04225913]
 Loss: 3994.062747456086, Parameters: [25.94520891  0.89186681  1.48990073 -0.03137339 -0.19999542  0.22216998
  3.94953071]
Epoch 201, Gradients: [ -0.71502723  -1.46208672  -0.19979461  -0.06243616   0.08686224
   1.47317461 -11.85213468]
 Loss: 3991.1293342523304, Parameters: [26.07163362  0.90448805  1.50215594 -0.03167833 -0.20084263  0.21303981
  4.16494505]
Epoch 301, Gradients: [ -0.63206679  -1.30166135  -0.15316611   0.01922782   0.0

optimization for vecchia approximation

In [3]:
coarse_factor = 4 # 16:2 8:4  4:16, i expect 2:64, 1:128
spline_instance = kernels.spline(epsilon = 0, coarse_factor=coarse_factor, nheads= 50, smooth = v, input_map= analysis_data_map, aggregated_data= aggregated_data, nns_map=nns_map, mm_cond_number=10)


optimizer, scheduler = spline_instance.optimizer_fun(params, lr=0.02, betas=(0.9, 0.99), eps=1e-8, step_size=100, gamma=0.2)  
out, epoch = spline_instance.fit_vecchia(params, optimizer,scheduler, epochs=500)

Epoch 1, Gradients: [  -7.84520081    1.62468763   18.552386    -24.41808612  220.31895698
 -973.27211035  -63.95389994]
 Loss: 4245.124948983874, Parameters: [ 2.47934437e+01  1.58452892e+00  1.71824777e+00  9.08850413e-03
 -1.07299447e-01  1.31037638e-01  2.71723866e+00]
Epoch 101, Gradients: [-2.44526084 11.09780925 16.36947514  6.72320832 -8.50745885 -1.22305276
 -7.63201109]
 Loss: 4114.927050663252, Parameters: [26.46990984  1.29531023  1.48314225  0.11630377 -0.18570608  0.53414916
  4.05745911]
Epoch 201, Gradients: [-2.12833194  9.23052533 13.55572248  6.81998545 -8.45476403  0.90502241
 -6.64227559]
 Loss: 4106.635305981222, Parameters: [26.77428674  1.24323377  1.45687867  0.09458664 -0.16938059  0.53490802
  4.31081679]
Epoch 301, Gradients: [-2.05531807  8.74883992 12.88462052  6.75949797 -8.25944442  0.95955198
 -6.41588328]
 Loss: 4104.559613694063, Parameters: [26.84243532  1.22711022  1.44865365  0.08680119 -0.16363596  0.53384385
  4.37275278]
Epoch 401, Gradients: [-

# Saved spline class (May 26, 2025)

In [ ]:
class spline(spatio_temporal_kernels):
    '''
    fit_cublic_spline() for each data shares the common locations. Even though the
    'distances' matrix is a function of parameters, we can make a common upper bound
    by putting range parameters 0.5, advections 0, beta 2.
    and we fit cubic_spline() for fixed smooth Matern model with range=1 and sigmasq=1.
    Essentially, we are approximating simple Matern model for v=1.
    
    Any change in parameters will be reflected through "distances" matrix. So,
    we define "distances" matrix for each epoch.
    
    '''
    def __init__(self, epsilon:float, coarse_factor_head:int, coarse_factor_cond:int, smooth:float, input_map: Dict[str, Any], aggregated_data:torch.Tensor, nns_map: np.ndarray, mm_cond_number:int):
        super().__init__(smooth, input_map, aggregated_data, nns_map, mm_cond_number)
        self.smooth = torch.tensor(smooth, dtype= torch.float64)
        
        self.epsilon = epsilon  # starting point for the spline fitting
        sample_params = [25, 0.5, 0.5, 0, 0, 2, 5] # just random nuumber to initialize spline
        sample_params = torch.tensor(sample_params, dtype=torch.float64, requires_grad=True)
        
        self.coarse_factor_head = coarse_factor_head
 
        self.coarse_factor_cond = coarse_factor_cond

        """
        Initialize the class with given parameters.
        Args:
            coarse_factor (int): Factor used for coarse-graining.
            smooth (float): Smooth parameter in Matern model.
            input_map (Dict[str, Any]): Dictionary containing input mappings.
            aggregated_data (torch.Tensor): Tensor containing aggregated data.
            nns_map (Dict[str, Any]): 2-d nd.array containing nearest neighbors mappings.
            mm_cond_number (int): Condition number for Vecchia approximation
        """

    def fit_cubic_spline(self, target_distances, coarse_factor:int=4):

        """
        Fit a natural cubic spline coefficients.

        Args:
            params (tuple): Parameters for the spline fitting.

        Returns:
            NaturalCubicSpline: The fitted spline object with coefficients.
        """

        def flat_distance_matrix(distances: torch.Tensor) -> torch.Tensor:
            n = distances.size(0)
            indices = torch.triu_indices(n, n, offset=1)
            upper_tri = distances[indices[0], indices[1]]
            unique_sorted = torch.unique(upper_tri, sorted=True)
            flat_distances = torch.cat([torch.tensor([0.0], device=unique_sorted.device), unique_sorted])
            max_distance = torch.max(flat_distances).clone().detach()
            len_distance_arr = len(flat_distances)
            
            return max_distance, len_distance_arr
        
        max_distance, len_distance_arr = flat_distance_matrix(target_distances)

        # fit_distances should be 1 d array to be used in natural_cubic_spline_coeffs
        fit_distances = torch.linspace(0, max_distance + 1e-6 , len_distance_arr// coarse_factor)
        non_zero_indices = fit_distances != 0
        out = torch.zeros_like(fit_distances, dtype= torch.float64)

        if torch.any(non_zero_indices):
            tmp = kv(self.smooth, torch.sqrt(fit_distances[non_zero_indices])).double().clone()
            out[non_zero_indices] = (1 * (2**(1-self.smooth)) / gamma(self.smooth) *
                                    (torch.sqrt(fit_distances[non_zero_indices]) ) ** self.smooth *
                                    tmp)
        out[~non_zero_indices] = 1

        # Compute spline coefficients. If input is tensor, so is output.
        # natural_cubic_spline_coeffs(t,x), t should be 1-d array (n,) and x should be (n,channels)
        # where channels reoresent number of features. out.unsquueze(1) makes (n,1).
        coeffs = natural_cubic_spline_coeffs(fit_distances, out.unsqueeze(1))
        # Create spline object
        spline = NaturalCubicSpline(coeffs)
        return spline


    def interpolate_cubic_spline(self, params:torch.Tensor, target_distances:torch.Tensor, spline_object) -> torch.Tensor:

        """
        Interpolate using the fitted cubic spline.
        Args:
            params (tuple): Parameters for the interpolation.
            target_distances (torch.Tensor): Distances to interpolate.
            spline_object (NaturalCubicSpline): The fitted spline object.

        Returns:
            torch.Tensor: Interpolated values.
        """
    
        sigmasq, _, _, _, _, _, nugget = params
        n = target_distances.size(0)
        indices = torch.triu_indices(n, n, offset=0)  # offset=0 to include diagonal

        # Evaluate spline only on upper triangle
        cov_upper = spline_object.evaluate(target_distances[indices[0], indices[1]])

        # Create empty matrix and fill upper triangle
        cov_matrix = torch.zeros_like(target_distances)

        # spline_object.evaluate return [N,1] 
        #print(cov_matrix.shape, cov_upper.shape, indices.shape, indices[0].shape)
        #print(indices)
        
        cov_matrix[indices[0], indices[1]] = cov_upper.view(-1)

        # Mirror to lower triangle
        cov_matrix = cov_matrix + cov_matrix.T - torch.diag(torch.diag(cov_matrix))

        # Apply scaling and nugget
        cov_matrix = cov_matrix * sigmasq
        cov_matrix = cov_matrix + torch.eye(n, dtype=torch.float64, device=cov_matrix.device) * nugget

        ''' 
        Before May26
        sigmasq, _, _, _, _, _, nugget = params
        cov_1d = spline_object.evaluate(target_distances)
        cov_matrix = cov_1d.reshape(target_distances.shape)
        cov_matrix = cov_matrix * sigmasq
        cov_matrix = cov_matrix + torch.eye(cov_matrix.shape[0], dtype=torch.float64) * nugget 
        '''
        return cov_matrix


    def full_likelihood_using_spline(self, params:torch.Tensor, input_data: torch.Tensor, y: torch.Tensor, target_distances:torch.Tensor, spline_object):
    
        cov_matrix = self.interpolate_cubic_spline(params, target_distances, spline_object)

        sign, log_det = torch.slogdet(cov_matrix)

        # if sign <= 0:
        #     raise ValueError("Covariance matrix is not positive definite")
        # Compute beta

        locs = input_data[:,:2]
        response = input_data[:,2]

        tmp1 = torch.matmul(locs.T, torch.linalg.solve(cov_matrix, locs))
        tmp2 = torch.matmul(locs.T, torch.linalg.solve(cov_matrix, response))
        beta = torch.linalg.solve(tmp1, tmp2)

        mu = torch.matmul(locs, beta)
        y_mu = response - mu
        quad_form = torch.matmul(y_mu, torch.linalg.solve(cov_matrix, y_mu))
        neg_log_lik = 0.5 * (log_det + quad_form)
        return  neg_log_lik

    def cov_structure_saver_using_spline(self, params: torch.Tensor) -> None:
        
        cov_map = defaultdict(lambda: defaultdict(dict))
        cut_line= self.nheads
        key_list = list(self.input_map.keys())

        for time_idx in range(0,3):
            current_array = self.input_map[key_list[time_idx]]

            # Use below when working on local computer to avoid singular matrix
            for index in range(cut_line, self.size_per_hour):
                current_row = current_array[index].reshape(1, -1)
                mm_neighbors = self.nns_map[index]
                past = list(mm_neighbors) 
                data_list = []

                if past:
                    data_list.append(current_array[past])

                if time_idx > 0:
                    one_hour_lag = self.input_map[key_list[time_idx - 1]]
                    data_list.append(one_hour_lag[past + [index], :])

                if time_idx > 1:
                    two_hour_lag = self.input_map[key_list[time_idx -2]]
                    data_list.append(two_hour_lag [past + [index], :])
                
                conditioning_data = torch.vstack(data_list) if data_list else torch.empty((0, current_row.shape[1]), dtype=torch.float64)
                aggregated_arr = torch.vstack((current_row, conditioning_data))
                locs = aggregated_arr[:, :2]

                target_distances_for_cond, non_zero_indices = self.precompute_coords_anisotropy(params, aggregated_arr,aggregated_arr)

                cond_spline_object = self.fit_cubic_spline(target_distances_for_cond, self.coarse_factor_cond )  # change here  
                cov_matrix = self.interpolate_cubic_spline(params, target_distances_for_cond, cond_spline_object)

                # if sign <= 0:
                #     raise ValueError("Covariance matrix is not positive definite")

                cov_yx = cov_matrix[0, 1:]
                sign, log_det = torch.slogdet(cov_matrix)
                tmp1 = torch.matmul(locs.T, torch.linalg.solve(cov_matrix, locs))
            
                # Mean and variance of y|x
                sigma = cov_matrix[0, 0]
                cov_xx = cov_matrix[1:, 1:]
                cov_xx_inv = torch.linalg.inv(cov_xx)
                cov_ygivenx = sigma - torch.matmul(cov_yx, torch.matmul(cov_xx_inv, cov_yx))
                cond_mean_tmp = torch.matmul(cov_yx, cov_xx_inv)
                log_det = torch.log(cov_ygivenx)
            
                cov_map[(time_idx,index)] = {
                    'tmp1': tmp1.clone().detach(),
                    'cov_xx_inv': cov_xx_inv.clone().detach(),
                    'cov_matrix': cov_matrix.clone().detach(),
                    'cov_ygivenx': cov_ygivenx.clone().detach(),
                    'cond_mean_tmp': cond_mean_tmp.clone().detach(),
                    'log_det': log_det.clone().detach(),
                    'locs': locs.clone().detach()
                }
        return cov_map

    def vecchia_nll_using_spline(self, params: torch.Tensor, cov_map:Dict[str,Any]) -> torch.Tensor:

        cut_line= self.nheads
        key_list = list(self.input_map.keys())
        neg_log_lik = 0.0
        heads = self.input_map[key_list[0]][:cut_line,:]

        for time_idx in range(1, len(self.input_map)):
            tmp = self.input_map[key_list[time_idx]][:cut_line,:]
            heads = torch.cat( (heads,tmp), dim=0)

        print(heads.shape )
        distances_heads, _ = self.precompute_coords_anisotropy(params, heads, heads)
        spline_object_head = self.fit_cubic_spline( distances_heads, self.coarse_factor_head)  # change here

        neg_log_lik += self.full_likelihood_using_spline(params, heads[:,:4], heads[:,2], distances_heads, spline_object_head)
    
        for time_idx in range(0,len(self.input_map)):
            current_np = self.input_map[key_list[time_idx]]

            for index in range(cut_line, self.size_per_hour):
                current_row = current_np[index].reshape(1, -1)
                current_y = current_row[0, 2]

                mm_neighbors = self.nns_map[index]
                past = list(mm_neighbors) 
                data_list = []

                if past:
                    data_list.append(current_np[past])  
                if time_idx < 2:
                    cov_matrix = cov_map[(time_idx,index)]['cov_matrix']
                    tmp1 = cov_map[(time_idx,index)]['tmp1']
                    cov_xx_inv = cov_map[(time_idx,index)]['cov_xx_inv']
                    cov_ygivenx = cov_map[(time_idx,index)]['cov_ygivenx']
                    cond_mean_tmp = cov_map[(time_idx,index)]['cond_mean_tmp']
                    log_det = cov_map[(time_idx,index)]['log_det']
                    locs = cov_map[(time_idx,index)]['locs']
                else:
                    cov_matrix = cov_map[(2,index)]['cov_matrix']
                    tmp1 = cov_map[(2,index)]['tmp1']
                    cov_xx_inv = cov_map[(2,index)]['cov_xx_inv']
                    cov_ygivenx = cov_map[(2,index)]['cov_ygivenx']
                    cond_mean_tmp = cov_map[(2,index)]['cond_mean_tmp']
                    log_det = cov_map[(2,index)]['log_det']
                    locs = cov_map[(2,index)]['locs']

                if time_idx >= 1:
                    one_hour_lag = self.input_map[key_list[time_idx - 1]]
                    past_conditioning_data = one_hour_lag[past + [index], :]
                    data_list.append(past_conditioning_data)
                
                if time_idx > 1:
                    two_hour_lag = self.input_map[key_list[time_idx - 2]]
                    past_conditioning_data = two_hour_lag[past + [index], :]
                    data_list.append(past_conditioning_data)
    
                if data_list:
                    conditioning_data = torch.vstack(data_list)
                else:
                    conditioning_data = torch.empty((0, current_row.shape[1]), dtype=torch.float64)

                aggregated_arr = torch.vstack((current_row, conditioning_data))
                aggregated_y = aggregated_arr[:, 2]

                cov_yx = cov_matrix[0, 1:]
                tmp2 = torch.matmul(locs.T, torch.linalg.solve(cov_matrix, aggregated_y))
                beta = torch.linalg.solve(tmp1, tmp2)
                mu = torch.matmul(locs, beta)
                mu_current = mu[0]
                mu_neighbors = mu[1:]
                
                # Mean and variance of y|x
                cond_mean = mu_current + torch.matmul(cond_mean_tmp, (aggregated_y[1:] - mu_neighbors))
                alpha = current_y - cond_mean
                quad_form = alpha**2 * (1 / cov_ygivenx)
                neg_log_lik += 0.5 * (log_det + quad_form)
        return neg_log_lik

    def compute_full_nll(self, params:torch.Tensor, aggregated_data, distances:torch.Tensor, spline_object): 
        nll = self.full_likelihood_using_spline( params, aggregated_data[:,:4], aggregated_data[:,2], distances, spline_object)
        return nll

    def compute_vecchia_nll(self, params:torch.Tensor): 
        cov_map = self.cov_structure_saver_using_spline(params)
        nll = self.vecchia_nll_using_spline(params, cov_map)
        return nll

    def optimizer_fun(self, params:torch.Tensor, lr:float =0.01, betas: tuple=(0.9, 0.8), eps:float=1e-8, step_size:int=40, gamma:float=0.5):
        optimizer = torch.optim.Adam([params], lr=lr, betas=betas, eps=eps)
        scheduler = StepLR(optimizer, step_size=step_size, gamma=gamma)  # Decrease LR by a factor of 0.1 every 10 epochs
        return optimizer, scheduler

    def run_full(self, params:torch.Tensor, aggregated_data, optimizer:torch.optim.Optimizer, scheduler:torch.optim.lr_scheduler, epochs:int=10 ):

        """
        Run the training loop for the full likelihood model.

        Args:
            params (torch.Tensor): Model parameters.
            optimizer (torch.optim.Optimizer): Optimizer for updating parameters.
            scheduler (torch.optim.lr_scheduler): Learning rate scheduler.
            epochs (int): Number of epochs to train.

        Returns:
            list: Final parameters and loss.
            int: Number of epochs run.
        """

        prev_loss= float('inf')
        # 1e-3: Faster convergence, slightly lower accuracy than 1e-4
        tol = 1e-3  # Convergence tolerance
        for epoch in range(epochs):  
            optimizer.zero_grad()  # Zero the gradients 
            distances, non_zero_indices = self.precompute_coords_anisotropy(params, aggregated_data[:,:4], aggregated_data[:,:4])
            spline_object = self.fit_cubic_spline( distances, self.coarse_factor_head)  # change here

            loss = self.compute_full_nll(params, aggregated_data, distances, spline_object)
            loss.backward()  # Backpropagate the loss

            # Gradient and Parameter Logging for every 10th epoch
            #if epoch % 10 == 0:
            #    print(f'Epoch {epoch+1}, Gradients: {params.grad.numpy()}\n Loss: {loss.item()}, Parameters: {params.detach().numpy()}')
            
            optimizer.step()  # Update the parameters
            scheduler.step()  # Update the learning rate

            # Convergence Check
            if abs(prev_loss - loss.item()) < tol:
                print(f"Converged at epoch {epoch}")
                print(f'Epoch {epoch+1}, : Loss: {loss.item()}, \n vecc Parameters: {params.detach().numpy()}')
                break

            prev_loss = loss.item()
        params = [torch.round(x*1000).detach().numpy()/1000 for x in params]
        loss = (torch.round(loss*1000)/1000).item()
        print(f'FINAL STATE: Epoch {epoch+1}, Loss: {loss}, \n vecc Parameters: {params}')
        return params + [loss], epoch

    def fit_vecchia(self, params:torch.Tensor, optimizer:torch.optim.Optimizer, scheduler:torch.optim.lr_scheduler, epochs:int=10 ):

        """
        Run the training loop for the full likelihood model.

        Args:
            params (torch.Tensor): Model parameters.
            optimizer (torch.optim.Optimizer): Optimizer for updating parameters.
            scheduler (torch.optim.lr_scheduler): Learning rate scheduler.
            epochs (int): Number of epochs to train.

        Returns:
            list: Final parameters and loss.
            int: Number of epochs run.
        """

        prev_loss= float('inf')
        # 1e-3: Faster convergence, slightly lower accuracy than 1e-4
        tol = 1e-3  # Convergence tolerance

        for epoch in range(epochs):  
            optimizer.zero_grad()  # Zero the gradients 
            # distance is a function of parameters
            # distances, non_zero_indices = self.precompute_coords_anisotropy(params, self.new_aggregated_data[:,:4], self.new_aggregated_data[:,:4])
            
            loss = self.compute_vecchia_nll(params)
            loss.backward()  # Backpropagate the loss

            # Gradient and Parameter Logging for every 10th epoch
            if epoch % 10 == 0:
                print(f'Epoch {epoch+1}, Gradients: {params.grad.numpy()}\n Loss: {loss.item()}, Parameters: {params.detach().numpy()}')
            
            # if epoch % 500 == 0:
            #     print(f'Epoch {epoch+1}, Gradients: {params.grad.numpy()}\n Loss: {loss.item()}, Parameters: {params.detach().numpy()}')
            
            optimizer.step()  # Update the parameters
            scheduler.step()  # Update the learning rate

            # Convergence Check
            if abs(prev_loss - loss.item()) < tol:
                print(f"Converged at epoch {epoch}")
                print(f'Epoch {epoch+1}, : Loss: {loss.item()}, \n vecc Parameters: {params.detach().numpy()}')
                break

            prev_loss = loss.item()
        params = [torch.round(x*1000).detach().numpy()/1000 for x in params]
        loss = (torch.round(loss*1000)/1000).item()
        print(f'FINAL STATE: Epoch {epoch+1}, Loss: {loss}, \n vecc Parameters: {params}')
        return params + [loss], epoch